# Wind and fluid drag  
In this notebook I perform some test calculation of the effect of having both wind and fluid drag (with a wind and fluid particle reynolds number) working on the particle. 

In [ ]:
# reload updated packages
%load_ext autoreload
%autoreload 2


# import needed packages


import numpy as np 
import cmath
import sys
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
sys.path.append('../src')
plt.style.use('../python_style_Meike.mplstyle')
import particle_characteristics_functions as pcf 

In [ ]:
name1 = 'test_everyting_{tres}hello'
name2 =name1.format(tres='daily_')
print(name2)

In [ ]:
def calculate_sigma(rho_p, rho_air, rho_water):
    delta_water = rho_water / rho_p 
    delta_air = rho_air / rho_p
    return (1-delta_air)/(delta_water-delta_air)

def calculate_h_air( B : float, radius : float):
    
    arg1 = 1-(2*B-1)**2
    arg = 1j*(arg1)**(1/2)+2*B-1
    # print(arg)
    phi = (arg)**(1/3)
    PHI = 1j*3**(1/2)/2 * (1/phi - phi) - 1/2 * (1/phi + phi) +1

    return PHI.real * radius

def calculate_projected_area(height: float, radius: float) -> float:
    return radius**2 * np.arccos(1 - height / radius) - (radius - height) * np.sqrt(radius**2 - (radius - height)**2)





In [ ]:
rho_air = 1.293 # kg m−3
rho_water = 1027 # kg/m3
mu_water = 1.41E-3 # Ns/m2 or Pa s #https://www.engineeringtoolbox.com/sea-water-properties-d_840.html at 10 deg
mu_air = 1.764E-5 # Ns/m2 or Pa s #https://www.engineeringtoolbox.com/air-absolute-kinematic-viscosity-d_601.html at 10 deg
d_in = 0.2 # m (estimated) dimater inner ring, needed to caluclate volume
d = 0.25 # m diameter outer ring
h = 0.041  # m (heigth drifter)
m = 0.905 # kg (mass drifter)
rho_p = pcf.density_drifter(diameter=d_in, heigth=h, mass=m)
sigma = calculate_sigma(rho_p=rho_p, rho_air =rho_air, rho_water = rho_water)
print(sigma)
print(rho_p/rho_water)

Buoyancy = rho_p/rho_water

In [ ]:
print(d/(mu_water/rho_water))
print(1.8e5)


In [ ]:
#it does not matter whether you use sigma or B
print(calculate_h_air(sigma,1))
print(calculate_h_air(rho_p/rho_water,1))

In [ ]:
# stokes relaxation time for nonzero Reynolds number for particle that partly sticks out of water
def tau_stokes_Rep_windage(rho_water, mu_water, mu_air, B, diameter, Rep_water, Rep_air):
    a = diameter/2 # radius
    c_Rep_water = pcf.factor_drag_white1991(Rep_water)
    c_Rep_air = pcf.factor_drag_white1991(Rep_air)
    gamma = (mu_air * c_Rep_air)/ (mu_water * c_Rep_water)
    
    h_air = calculate_h_air(B,a)
    A_air = calculate_projected_area(h_air, a)
    
    A_water = calculate_projected_area(diameter-h_air, a)
    # print(A_water/(np.pi*(diameter/2)**2))
    A_sphere = np.pi *  a**2
    Afactor = (A_water + gamma*A_air) / A_sphere
    tau =  a**2 *(1-1/6*h_air/a) * B *rho_water/ (3*Afactor *  mu_water * c_Rep_water)
    return tau

def windage_factor(B,diameter, mu_water, mu_air, Rep_water, Rep_air):
    a= diameter/2
    c_Rep_water = pcf.factor_drag_white1991(Rep_water)
    c_Rep_air = pcf.factor_drag_white1991(Rep_air)
    gamma = (mu_air * c_Rep_air)/ (mu_water * c_Rep_water)
    h_air = calculate_h_air(B,a)
    A_air = calculate_projected_area(h_air, a)
    

    A_air = calculate_projected_area(h_air, a)
    
    A_water = calculate_projected_area(diameter-h_air, a)
    return gamma*A_air/ (A_water + gamma*A_air)



In [ ]:
Rep_water = 0 
Rep_air = 0
# at least if B = 1 they the tau drag with rep and windage reduces to the stokes drag for Rep=0 for completely submerged particle
tau = tau_stokes_Rep_windage(rho_water=rho_water, mu_water = mu_water, mu_air = mu_air, B = 1,diameter = d, Rep_water = Rep_water, Rep_air = Rep_air ) 
print(tau/3600)
tau_stokes = pcf.stokes_relaxation_time(diameter = d, kinematic_viscosity=mu_water/rho_water,buoyancy=1)
print(tau_stokes/3600)

In [ ]:
# if small reynoldsnumber tau is bigger for partly stikcing out particle (intresting..)

Rep_water = 0 
Rep_air = 0
tau = tau_stokes_Rep_windage(rho_water=rho_water, mu_water = mu_water, mu_air = mu_air, B = Buoyancy,diameter = d, Rep_water = Rep_water, Rep_air = Rep_air ) 
print(tau/3600)
tau_stokes = pcf.stokes_relaxation_time(diameter = d, kinematic_viscosity=mu_water/rho_water,buoyancy=Buoyancy)
print(tau_stokes/3600)

In [ ]:
Rep_air = pcf.Re_particle(Uslip=4,kinematic_viscosity=mu_air/rho_air, diameter=d)
Rep_water = pcf.Re_particle(Uslip=0.1,kinematic_viscosity=mu_water/rho_water,diameter=d)
print(Rep_air)
print(Rep_water)
alpha = windage_factor(B=Buoyancy,diameter=d, mu_water=mu_water, mu_air=mu_air, Rep_water=Rep_air, Rep_air= Rep_water)
print(alpha)

In [ ]:
pcf.Re_particle(Uslip=1,kinematic_viscosity=mu_water/rho_water,diameter=d)

In [ ]:
# now from flume experiment
d = 0.22 # diameter is 22 cm
B = 0.5 # values of 1/B are 3.9, 2, 1.2, and 2 (here we use 2)
u_p = 0.15
u_air = 1.10 # m/s
u_w = 0.14 # m/s
Rep_air = pcf.Re_particle(Uslip=np.abs(u_air-u_p),kinematic_viscosity=mu_air/rho_air, diameter=d)
Rep_water = pcf.Re_particle(Uslip=np.abs(u_w-u_p),kinematic_viscosity=mu_water/rho_water,diameter=d)
print(Rep_air)
print(Rep_water)
alpha = windage_factor(B=Buoyancy,diameter=d, mu_water=mu_water, mu_air=mu_air, Rep_water=Rep_air, Rep_air= Rep_water)
print(alpha)
tau = tau_stokes_Rep_windage(rho_water=rho_water, mu_water = mu_water, mu_air = mu_air, B = Buoyancy,diameter = d, Rep_water = Rep_water, Rep_air = Rep_air ) 
print(tau) 

# very high reynolds number which is a bit weird (or maybe not but then tau reduces to 0 if you do this? ) 
# :( 

In [ ]:
fig, ax = plt.subplots()
uslip=np.logspace(-4,1,50)
nu_water = mu_water/rho_water
print(uslip[0])
# ax.plot(uslip,pcf.Re_particle(Uslip=uslip,kinematic_viscosity=mu_water/rho_water,diameter=d),'--o')
ax.set_yscale('log')
ax.set_xscale('log')
# fdrag = 3*np.pi*nu_water*uslip*pcf.factor_drag_white1991(pcf.Re_particle(Uslip=uslip,kinematic_viscosity=mu_water/rho_water,diameter=d))
fdrag2 = 3*np.pi*nu_water*uslip*pcf.factor_drag_morrison2013(pcf.Re_particle(Uslip=uslip,kinematic_viscosity=mu_water/rho_water,diameter=d))


# ax.plot(uslip,fdrag)
ax.plot(uslip,fdrag2,color='firebrick')
ax2 = ax.twiny()
ax2.plot(pcf.Re_particle(Uslip=uslip,kinematic_viscosity=mu_water/rho_water,diameter=d),fdrag2,color='firebrick')
ax2.set_xscale('log')
ax.set_ylabel('|Fdrag|')
ax.set_xlabel('Uslip')
ax2.set_xlabel('Rep')

In [ ]:
0.25*0.25

In [ ]:
tau_diff = 0.25**2/(mu_water/rho_water) 

In [ ]:
0.125**2

In [ ]:
tau_diff * (0.502/10000000 + 0.123)**2 

In [ ]:
'NWES_start2023_09_01_T0719h_Rep_0000B0680_tau2994_cor_True_gradient_False.geojson'
'NWES_start2023_09_01_T0719h_Rep_0000_cor_True_gradient_True.geojson'